## Feature Ideas

1. Airline recode
    1. size of airlines # flights  
    2. size of airlines company size  
    3. size of airlines profitability  
    
    
2. Covid
    1. new cases in london by day
    2. new cases in the UK by day (maybe different decision making bases on one and not the other?)  
    3. If we can get carrier country - we can using 1.2 Webscrape_IATA_codes.ipynb then we can get daily covid cases at destination too.



3. Worker strikes
    1. web crawler on the airline union websites - matches of the word 'strike' in the last 2 week period (time needed for organisation)   
    2. other industrial actions  


4. Twitter
    1. Sentiment analysis on Tweets __from__ heathrow leadership accounts  
    2. Sentiment analysis on Tweets __about__ heathrow leadership using hastags  
    3. aviation authority - bag of keywords  


5. Weather
    1. daily api hit for forecasts on flight day __(careful not to include future information)__ - score forecast events on a scale of 'disruptiveness' 


6. Polical/civil/security issues
    1. webscrape of news headlines. Either looking for keywords 'airport' OR sentiment score or both.  


7. technical faults/disruptions
    1. public airport information - websites  
    2. Boeing/Airbus announcments - faults  


8. World Bank - airline passengers API
    1. Passengers from PREVIOUS month increase/decrease % - will a recent decrease of passengers result in more cancellations?


In [10]:
#Libraries
import os
import pandas as pd
import numpy as np
import datetime as dt
import fastparquet as fp
from dateutil import parser
import calendar
import pickle

In [18]:
#import data
data = pd.read_pickle('C:\\Users\\ellio\\Desktop\\Python\\scripts\\ML_flight_cancellation_prediction\\datasets\\3-data_for_features.pickle')
data.dtypes

type                                 object
status                               object
departure_iatacode                   object
departure_icaocode                   object
departure_terminal                   object
arrival_terminal                     object
airline_name                         object
flight_number                        object
flight_iatanumber                    object
codeshared_airline_name              object
codeshared_flight_number             object
departure_gate                       object
departure_estimatedrunway            object
departure_actualrunway               object
departure_scheduledtime      datetime64[ns]
arrival_scheduledtime        datetime64[ns]
departure_delay              datetime64[ns]
departure_estimatedtime      datetime64[ns]
departure_actualtime         datetime64[ns]
weekday                              object
time_of_day                          object
date                                 object
cancelled_outcome               

In [19]:
# Filter airline_name == empty, blanks and spaces.
filtered_df = data[(data.airline_name != "empty")|(data.airline_name != "")|(data.airline_name != " ")]
filtered_df.head(3)

,type,status,departure_iatacode,departure_icaocode,departure_terminal,arrival_terminal,airline_name,flight_number,flight_iatanumber,codeshared_airline_name,...,departure_actualrunway,departure_scheduledtime,arrival_scheduledtime,departure_delay,departure_estimatedtime,departure_actualtime,weekday,time_of_day,date,cancelled_outcome
0,departure,unknown,lhr,egll,2,5,thai airways international,6225,tg6225,sas,...,None,2020-05-15 06:40:00,2020-05-15 10:05:00,NaT,NaT,NaT,Friday,06:40:00,2020-05-15,0
1,departure,unknown,lhr,egll,2,5,sas,1530,sk1530,None,...,None,2020-05-15 06:40:00,2020-05-15 10:05:00,NaT,NaT,NaT,Friday,06:40:00,2020-05-15,0
2,departure,unknown,lhr,egll,2,3,sas,500,sk500,None,...,None,2020-05-15 06:45:00,2020-05-15 09:35:00,NaT,NaT,NaT,Friday,06:45:00,2020-05-15,0


# insert other features above here. ^^ From here we are going to drop columns and encode.

In [20]:
drop_cols = ['type', 'status', 'departure_iatacode', 'departure_icaocode', 
             'flight_number', 'flight_iatanumber', 'departure_actualrunway', 
            'arrival_scheduledtime', 'departure_delay', 
             'departure_estimatedtime', 'departure_actualtime', 'date', 'departure_estimatedrunway']

model_cols_df = filtered_df.drop(columns = drop_cols)

model_cols_df.head(3)

,departure_terminal,arrival_terminal,airline_name,codeshared_airline_name,codeshared_flight_number,departure_gate,departure_scheduledtime,weekday,time_of_day,cancelled_outcome
0,2,5,thai airways international,sas,1530,None,2020-05-15 06:40:00,Friday,06:40:00,0
1,2,5,sas,None,None,None,2020-05-15 06:40:00,Friday,06:40:00,0
2,2,3,sas,None,None,None,2020-05-15 06:45:00,Friday,06:45:00,0


In [21]:
# change codeshared_airline_name to 'is_codeshare'

model_cols_df['is_codeshare'] = np.where(model_cols_df['codeshared_airline_name'].isnull() , 0, 1)

model_cols_df.drop(['codeshared_flight_number', 'codeshared_airline_name'], axis=1, inplace = True)

model_cols_df.dtypes

short_df = model_cols_df

In [22]:
short_df['hour_of_day'] = short_df.departure_scheduledtime.dt.hour
short_df.head(3)

,departure_terminal,arrival_terminal,airline_name,departure_gate,departure_scheduledtime,weekday,time_of_day,cancelled_outcome,is_codeshare,hour_of_day
0,2,5,thai airways international,None,2020-05-15 06:40:00,Friday,06:40:00,0,1,6
1,2,5,sas,None,2020-05-15 06:40:00,Friday,06:40:00,0,0,6
2,2,3,sas,None,2020-05-15 06:45:00,Friday,06:45:00,0,0,6


In [23]:
#turn time_of_day into 4 segments redeye, morning, afternoon, evening

# redeye - 10pm to 0559

# morning - 6am to 1159

#afternoon - midday to 1759

#evening - 6pm to 2159

# Create the dictionary 
conditions = [
    (short_df['hour_of_day'] >= 22) & (short_df['hour_of_day'] <= 24),
    (short_df['hour_of_day'] >= 0) & (short_df['hour_of_day'] <= 6),
    (short_df['hour_of_day'] >= 7) & (short_df['hour_of_day'] < 12),
    (short_df['hour_of_day'] >= 12) & (short_df['hour_of_day'] <= 18),
    (short_df['hour_of_day'] >= 19) & (short_df['hour_of_day'] <= 22)]

values = ['redeye_flight', 'redeye_flight','morning_flight', 'afternoon_flight', 'evening_flight']
  
short_df['flight_tier'] = np.select(conditions, values)

# display updated DataFrame

short_df.head()

,departure_terminal,arrival_terminal,airline_name,departure_gate,departure_scheduledtime,weekday,time_of_day,cancelled_outcome,is_codeshare,hour_of_day,flight_tier
0,2,5,thai airways international,None,2020-05-15 06:40:00,Friday,06:40:00,0,1,6,redeye_flight
1,2,5,sas,None,2020-05-15 06:40:00,Friday,06:40:00,0,0,6,redeye_flight
2,2,3,sas,None,2020-05-15 06:45:00,Friday,06:45:00,0,0,6,redeye_flight
3,2,None,singapore airlines,None,2020-05-15 07:05:00,Friday,07:05:00,0,1,7,morning_flight
4,2,None,sas,None,2020-05-15 07:05:00,Friday,07:05:00,0,0,7,morning_flight


In [24]:
#check new feature for NA
set(short_df['flight_tier'])

{'afternoon_flight', 'evening_flight', 'morning_flight', 'redeye_flight'}

In [25]:
short_df.drop(columns = ['time_of_day'])

,departure_terminal,arrival_terminal,airline_name,departure_gate,departure_scheduledtime,weekday,cancelled_outcome,is_codeshare,hour_of_day,flight_tier
0,2,5,thai airways international,None,2020-05-15 06:40:00,Friday,0,1,6,redeye_flight
1,2,5,sas,None,2020-05-15 06:40:00,Friday,0,0,6,redeye_flight
2,2,3,sas,None,2020-05-15 06:45:00,Friday,0,0,6,redeye_flight
3,2,None,singapore airlines,None,2020-05-15 07:05:00,Friday,0,1,7,morning_flight
4,2,None,sas,None,2020-05-15 07:05:00,Friday,0,0,7,morning_flight
...,...,...,...,...,...,...,...,...,...,...
139287,2,a,virgin atlantic,None,2020-12-25 22:30:00,Friday,1,0,22,redeye_flight
139288,None,None,virgin atlantic,None,2020-12-25 14:45:00,Friday,0,0,14,afternoon_flight
139289,None,None,virgin atlantic,None,2020-12-25 15:00:00,Friday,0,0,15,afternoon_flight
139290,None,None,lufthansa,None,2020-12-25 14:15:00,Friday,0,0,14,afternoon_flight


## Encoding airlines - % of Heathrow flights - rank from 1-5. 124 unique carriers

In [53]:
#define crosstab
tab = pd.crosstab(index=short_df['airline_name'], columns='Per_of_all_H_flights')

#find proportions 
freq_tab = tab/tab.sum()

sorted_freq_tab = pd.DataFrame(freq_tab.sort_values(by = 'Per_of_all_H_flights', ascending = False)).reset_index()

sorted_freq_tab['rank'] = sorted_freq_tab['Per_of_all_H_flights'].rank(ascending = False)

joined_rank_df = pd.merge(short_df, sorted_freq_tab, how="left")

joined_rank_df.head(3)

,departure_terminal,arrival_terminal,airline_name,departure_gate,departure_scheduledtime,weekday,time_of_day,cancelled_outcome,is_codeshare,hour_of_day,flight_tier,Per_of_all_H_flights,rank
0,2,5,thai airways international,None,2020-05-15 06:40:00,Friday,06:40:00,0,1,6,redeye_flight,0.000337,95.0
1,2,5,sas,None,2020-05-15 06:40:00,Friday,06:40:00,0,0,6,redeye_flight,0.008629,26.0
2,2,3,sas,None,2020-05-15 06:45:00,Friday,06:45:00,0,0,6,redeye_flight,0.008629,26.0


In [62]:
# encode the values - leave in Per of heathrow flights for a linear and non-linear feature of the same variable
conditions = [
    (joined_rank_df['rank'] < 21),
    (joined_rank_df['rank'] >= 21) & (joined_rank_df['rank']<=50),
    (joined_rank_df['rank'] >= 51) & (joined_rank_df['rank']<=100),
    (joined_rank_df['rank'] > 100)]

values = ['freq_airline_top20', 'freq_airline_21_50','freq_airline_51_100', 'freq_airline_above100']

joined_rank_df['airline_HR_freq'] = np.select(conditions, values)

joined_rank_df = joined_rank_df.drop(columns = ['rank', 'airline_name', 'time_of_day', 'departure_scheduledtime', 'departure_gate'])

In [63]:
joined_rank_df.to_parquet('C:\\Users\\ellio\\Desktop\\Python\\scripts\\ML_flight_cancellation_prediction\\datasets\\4-Dataset_for_model.parquet.gzip',
               compression='gzip')